In [56]:
import pandas as pd
import xgboost as xgb
import numpy as np
import random
from sportsreference.ncaab.teams import Teams
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
teams = Teams(year = 2017)

In [4]:
teams_df = teams.dataframes

In [5]:
teams_df.head(10)

,abbreviation,assist_percentage,assists,away_losses,away_wins,block_percentage,blocks,conference,conference_losses,conference_wins,...,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,win_percentage,wins
ABILENE-CHRISTIAN,ABILENE-CHRISTIAN,56.6,408,11,4,8.5,83,southland,11,7,...,47.1,904,0.554,18.5,412,1005,0.511,514,0.448,13
AIR-FORCE,AIR-FORCE,60.5,494,11,0,5.7,57,mwc,14,4,...,50.3,1149,0.544,15.0,390,1148,0.482,553,0.364,12
AKRON,AKRON,54.5,517,6,7,8.6,118,mac,4,14,...,51.2,1246,0.580,15.0,422,1051,0.555,583,0.750,27
ALABAMA-AM,ALABAMA-AM,51.9,326,18,1,4.4,50,swac,16,2,...,46.4,914,0.487,17.3,384,1102,0.456,503,0.069,2
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,60.2,503,8,4,13.2,160,cusa,9,9,...,52.4,1188,0.556,17.3,444,1226,0.515,632,0.515,17
ALABAMA-STATE,ALABAMA-STATE,41.7,300,17,1,8.0,98,swac,12,6,...,50.4,1188,0.478,18.6,488,1160,0.447,519,0.258,8
ALABAMA,ALABAMA,48.2,390,6,5,11.6,141,sec,8,10,...,53.8,1327,0.515,17.3,474,1199,0.487,584,0.559,19
ALBANY-NY,ALBANY-NY,49.0,444,8,9,5.6,60,america-east,6,10,...,55.5,1257,0.555,15.6,416,1425,0.502,715,0.600,21
ALCORN-STATE,ALCORN-STATE,53.1,425,11,6,9.2,115,swac,5,13,...,51.4,1243,0.515,17.3,452,1197,0.483,578,0.563,18
AMERICAN,AMERICAN,49.0,339,11,4,6.9,73,patriot,13,5,...,46.6,868,0.522,18.3,396,1013,0.517,524,0.267,8


In [6]:
team_names = teams_df[['abbreviation']]

In [7]:
team_names.head()

,abbreviation
ABILENE-CHRISTIAN,ABILENE-CHRISTIAN
AIR-FORCE,AIR-FORCE
AKRON,AKRON
ALABAMA-AM,ALABAMA-AM
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM


In [ ]:
#Only needed this on first run
#team_names.to_csv(r'data/team_names.csv')
#teams_df.to_csv(r'data/team_data.csv')

In [8]:
games = pd.read_csv(r'data/TourneyCompactResults.csv')

In [9]:
games[['Season','WTeamID','LTeamID']].loc[games['Season'] == 2017].head()

,Season,WTeamID,LTeamID
2050,2017,1243,1448
2051,2017,1291,1309
2052,2017,1413,1300
2053,2017,1425,1344
2054,2017,1112,1315


In [10]:
games = games[['Season','WTeamID','LTeamID']]
games['Winner'] = games['WTeamID']
games = games.rename(columns = {'WTeamID' : 'Team_A', 'LTeamID': 'Team_B'})
games.head()

,Season,Team_A,Team_B,Winner
0,1985,1116,1234,1116
1,1985,1120,1345,1120
2,1985,1207,1250,1207
3,1985,1229,1425,1229
4,1985,1242,1325,1242


In [12]:
i = 0
games['Team_X'] = 0
games['Team_Y'] = 0
while i < len(games):
    games['Team_X'][i] = random.choice(([games['Team_A'][i],games['Team_B'][i]]))
    i += 1
games['Team_Y'] = np.where(games['Team_X'] == games['Team_A'], games['Team_B'], games['Team_A'])
games['Result'] = np.where(games['Winner'] == games['Team_X'], 'X', 'Y')
games = games.drop(columns = ['Team_A','Team_B'], axis = 1)

In [13]:
games.head(25)

,Season,Winner,Team_X,Team_Y,Result
0,1985,1116,1116,1234,X
1,1985,1120,1345,1120,Y
2,1985,1207,1250,1207,Y
3,1985,1229,1229,1425,X
4,1985,1242,1242,1325,X
5,1985,1246,1246,1449,X
6,1985,1256,1338,1256,Y
7,1985,1260,1260,1233,X
8,1985,1314,1292,1314,Y
9,1985,1323,1333,1323,Y


In [14]:
team_data = teams_df[['abbreviation','assist_percentage','block_percentage','effective_field_goal_percentage','field_goal_percentage','free_throw_attempt_rate','free_throw_percentage','free_throws_per_field_goal_attempt','net_rating','offensive_rating','offensive_rebound_percentage','opp_assist_percentage','opp_block_percentage','opp_effective_field_goal_percentage','opp_field_goal_percentage','opp_free_throw_attempt_rate','opp_free_throw_percentage','opp_free_throws_per_field_goal_attempt','opp_offensive_rating','opp_offensive_rebound_percentage','opp_steal_percentage','opp_three_point_attempt_rate','opp_three_point_field_goal_percentage','opp_total_rebound_percentage','opp_true_shooting_percentage','opp_two_point_field_goal_percentage','pace','simple_rating_system','steal_percentage','strength_of_schedule','three_point_attempt_rate','three_point_field_goal_percentage','total_rebound_percentage','true_shooting_percentage','turnover_percentage','two_point_field_goal_percentage','win_percentage']]

In [15]:
games_2017 = games.loc[games['Season'] == 2017]
games_2017.head()

,Season,Winner,Team_X,Team_Y,Result
2050,2017,1243,1243,1448,X
2051,2017,1291,1291,1309,X
2052,2017,1413,1300,1413,Y
2053,2017,1425,1425,1344,X
2054,2017,1112,1112,1315,X


In [16]:
team_abb = pd.read_csv(r'data/team_names.csv')

In [17]:
team_abb = team_abb[['abbreviation','Team_Id']]

In [18]:
team_data_abb = team_data.merge(team_abb, left_on = 'abbreviation', right_on = 'abbreviation', how = 'left').dropna()
team_data_abb['Team_Id'] = team_data_abb['Team_Id'].astype(np.int64)

In [19]:
games_2017_a = games_2017.merge(team_data_abb, left_on = 'Team_X', right_on = 'Team_Id', how = 'left')
games_2017_b = games_2017_a.merge(team_data_abb, left_on = 'Team_Y', right_on = 'Team_Id', how = 'left')
games_2017_b = games_2017_b.dropna()
print(games_2017_b.head(15))

    Season  Winner  Team_X  Team_Y Result          abbreviation_x  \
0     2017    1243    1243    1448      X            KANSAS-STATE   
1     2017    1291    1291    1309      X          MOUNT-ST-MARYS   
2     2017    1413    1300    1413      Y  NORTH-CAROLINA-CENTRAL   
3     2017    1425    1425    1344      X     SOUTHERN-CALIFORNIA   
4     2017    1112    1112    1315      X                 ARIZONA   
5     2017    1139    1457    1139      Y                WINTHROP   
6     2017    1196    1190    1196      Y    EAST-TENNESSEE-STATE   
7     2017    1199    1195    1199      Y      FLORIDA-GULF-COAST   
8     2017    1211    1355    1211      Y      SOUTH-DAKOTA-STATE   
9     2017    1235    1235    1305      X              IOWA-STATE   
10    2017    1292    1278    1292      Y               MINNESOTA   
11    2017    1321    1435    1321      Y              VANDERBILT   
12    2017    1323    1343    1323      Y               PRINCETON   
13    2017    1345    1436    1345

In [20]:
games_2017_b['Winner'] = np.where(games_2017_b['Winner'] == games_2017_b['Team_X'], games_2017_b['abbreviation_x'], games_2017_b['abbreviation_y'])
games_2017_b['Team_X'] = games_2017_b['abbreviation_x']
games_2017_b['Team_Y'] = games_2017_b['abbreviation_y']

In [21]:
games_2017_b.dtypes

Season                                        int64
Winner                                       object
Team_X                                       object
Team_Y                                       object
Result                                       object
abbreviation_x                               object
assist_percentage_x                         float64
block_percentage_x                          float64
effective_field_goal_percentage_x           float64
field_goal_percentage_x                     float64
free_throw_attempt_rate_x                   float64
free_throw_percentage_x                     float64
free_throws_per_field_goal_attempt_x        float64
net_rating_x                                float64
offensive_rating_x                          float64
offensive_rebound_percentage_x              float64
opp_assist_percentage_x                     float64
opp_block_percentage_x                      float64
opp_effective_field_goal_percentage_x       float64
opp_field_go

In [22]:
print(games_2017_b.head(15))

    Season               Winner                  Team_X  \
0     2017         KANSAS-STATE            KANSAS-STATE   
1     2017       MOUNT-ST-MARYS          MOUNT-ST-MARYS   
2     2017     CALIFORNIA-DAVIS  NORTH-CAROLINA-CENTRAL   
3     2017  SOUTHERN-CALIFORNIA     SOUTHERN-CALIFORNIA   
4     2017              ARIZONA                 ARIZONA   
5     2017               BUTLER                WINTHROP   
6     2017              FLORIDA    EAST-TENNESSEE-STATE   
7     2017        FLORIDA-STATE      FLORIDA-GULF-COAST   
8     2017              GONZAGA      SOUTH-DAKOTA-STATE   
9     2017           IOWA-STATE              IOWA-STATE   
10    2017     MIDDLE-TENNESSEE               MINNESOTA   
11    2017         NORTHWESTERN              VANDERBILT   
12    2017           NOTRE-DAME               PRINCETON   
13    2017               PURDUE                 VERMONT   
14    2017       SAINT-MARYS-CA          SAINT-MARYS-CA   

                   Team_Y Result          abbreviation_

In [47]:
ml_input = games_2017_b.drop(columns = ['Season','Winner','Team_X','Team_Y','abbreviation_x','abbreviation_y','Team_Id_x','Team_Id_y'], axis = 1)
ml_input.dtypes

Result                                       object
assist_percentage_x                         float64
block_percentage_x                          float64
effective_field_goal_percentage_x           float64
field_goal_percentage_x                     float64
free_throw_attempt_rate_x                   float64
free_throw_percentage_x                     float64
free_throws_per_field_goal_attempt_x        float64
net_rating_x                                float64
offensive_rating_x                          float64
offensive_rebound_percentage_x              float64
opp_assist_percentage_x                     float64
opp_block_percentage_x                      float64
opp_effective_field_goal_percentage_x       float64
opp_field_goal_percentage_x                 float64
opp_free_throw_attempt_rate_x               float64
opp_free_throw_percentage_x                 float64
opp_free_throws_per_field_goal_attempt_x    float64
opp_offensive_rating_x                      float64
opp_offensiv

In [26]:
ml_input.to_csv(r'Data/ml_input.csv')

In [48]:
X_all = ml_input.drop(['Result'],1)
y_all = ml_input['Result']

In [49]:
cols = [['assist_percentage_x','block_percentage_x','effective_field_goal_percentage_x','field_goal_percentage_x','free_throw_attempt_rate_x','free_throw_percentage_x','free_throws_per_field_goal_attempt_x','net_rating_x','offensive_rating_x','offensive_rebound_percentage_x','opp_assist_percentage_x','opp_block_percentage_x','opp_effective_field_goal_percentage_x','opp_field_goal_percentage_x','opp_free_throw_attempt_rate_x','opp_free_throw_percentage_x','opp_free_throws_per_field_goal_attempt_x','opp_offensive_rating_x','opp_offensive_rebound_percentage_x','opp_steal_percentage_x','opp_three_point_attempt_rate_x','opp_three_point_field_goal_percentage_x','opp_total_rebound_percentage_x','opp_true_shooting_percentage_x','opp_two_point_field_goal_percentage_x','pace_x','simple_rating_system_x','steal_percentage_x','strength_of_schedule_x','three_point_attempt_rate_x','three_point_field_goal_percentage_x','total_rebound_percentage_x','true_shooting_percentage_x','turnover_percentage_x','two_point_field_goal_percentage_x','win_percentage_x','assist_percentage_y','block_percentage_y','effective_field_goal_percentage_y','field_goal_percentage_y','free_throw_attempt_rate_y','free_throw_percentage_y','free_throws_per_field_goal_attempt_y','net_rating_y','offensive_rating_y','offensive_rebound_percentage_y','opp_assist_percentage_y','opp_block_percentage_y','opp_effective_field_goal_percentage_y','opp_field_goal_percentage_y','opp_free_throw_attempt_rate_y','opp_free_throw_percentage_y','opp_free_throws_per_field_goal_attempt_y','opp_offensive_rating_y','opp_offensive_rebound_percentage_y','opp_steal_percentage_y','opp_three_point_attempt_rate_y','opp_three_point_field_goal_percentage_y','opp_total_rebound_percentage_y','opp_true_shooting_percentage_y','opp_two_point_field_goal_percentage_y','pace_y','simple_rating_system_y','steal_percentage_y','strength_of_schedule_y','three_point_attempt_rate_y','three_point_field_goal_percentage_y','total_rebound_percentage_y','true_shooting_percentage_y','turnover_percentage_y','two_point_field_goal_percentage_y','win_percentage_y']]
for col in cols:
    X_all[col] = scale(X_all[col])

In [52]:
len(X_all)

67

In [118]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,
                                                   test_size = 15,
                                                   random_state = 2,
                                                   stratify = y_all)

In [128]:
clf_A = LogisticRegression(random_state = 64)
clf_B = SVC(random_state = 64, kernel = 'rbf')
clf_C = xgb.XGBClassifier(seed = 64)

train_predict(clf_A, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_B, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_C, X_train, y_train, X_test, y_test)
print('')

Trained model LogisticRegression in 0.0020101070404052734  seconds
Made predictions in 0.002008199691772461 seconds.
F1 score and accuracy score for training set: 1.0  ,  1.0
Made predictions in 0.0 seconds.
F1 score and accuracy score for test set: 0.6153846153846153  ,  0.6666666666666666

Trained model SVC in 0.0010027885437011719  seconds
Made predictions in 0.0010025501251220703 seconds.
F1 score and accuracy score for training set: 0.9523809523809523  ,  0.9615384615384616
Made predictions in 0.0010030269622802734 seconds.
F1 score and accuracy score for test set: 0.2857142857142857  ,  0.6666666666666666

Trained model XGBClassifier in 0.05767035484313965  seconds
Made predictions in 0.0020034313201904297 seconds.
F1 score and accuracy score for training set: 1.0  ,  1.0
Made predictions in 0.0020058155059814453 seconds.
F1 score and accuracy score for test set: 0.6  ,  0.7333333333333333



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [127]:
#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    time_taken = end - start
    
    # Print the results
    print('Trained model', clf.__class__.__name__, 'in', time_taken, ' seconds')

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    time_taken = end - start
    print("Made predictions in", time_taken, "seconds.")
    
    return f1_score(target, y_pred, pos_label='X'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    #print("Training something using a training set size of %d. . .").format(len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print("F1 score and accuracy score for training set:", f1, " , ", acc)
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set:", f1, " , ", acc)

In [130]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer


# TODO: Create the parameters list you wish to tune
parameters = { 'learning_rate' : [0.1],
               'n_estimators' : [40],
               'max_depth': [3],
               'min_child_weight': [3],
               'gamma':[0.4],
               'subsample' : [0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             }  

# TODO: Initialize the classifier
clf = xgb.XGBClassifier(seed=2)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='X')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
f1, acc = predict_labels(clf, X_train, y_train)
print("F1 score and accuracy score for training set:", f1, " , ", acc)
    
f1, acc = predict_labels(clf, X_test, y_test)
print("F1 score and accuracy score for test set:", f1, " , ", acc)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
       n_estimators=40, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=1e-05,
       reg_lambda=1, scale_pos_weight=1, seed=2, silent=True,
       subsample=0.8)
Made predictions in 0.001970052719116211 seconds.
F1 score and accuracy score for training set: 0.8717948717948718  ,  0.9038461538461539
Made predictions in 0.002006053924560547 seconds.
F1 score and accuracy score for test set: 0.4444444444444444  ,  0.6666666666666666


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Retur